# Example of Output
## Detection
Output of the detection module of a volume:

```
[
  [[zs1, ys1, xs1, ze1, ye1, xe1], obj_score1, class1_score1, class2_score1],
  [[zs2, ys2, xs2, ze2, ye2, xe2], obj_score2, class1_score2, class2_score2],
  ...
]
```

In the format above, xs, ys, and zs represents starts of the bounding box, xe, ye and ze represents ends of the bounding box. "obj_score" represents the confidence of this bounding box, "class1_score" represents the probability of this bounding box as the first class (intracranial aneurysm in this challenge), and "class2_score" represents the probability of this bounding box as the second class (stenosis). Note that the sum of probabilities of two classes should be 1.

When evaluating each class of detection, the output will be processed using the following code to adapt the detection metrics code.

In [1]:
import numpy as np
def convert_output_to_detection(output):
    """
    Convert the output to the format of the detection.
    :param output: {img_id: [bbox]}
    :param wanted_class_num: 1 or 2
    :return: {img_id: [(bbox, score)]}
    """
    detection = {}
    for img_id, bbox_list in output.items():
        detection[img_id] = []
        for bbox in bbox_list:
            if bbox[2] >= bbox[3]:
                class_name = 'IA'
            else:
                class_name = 'stenosis'
            detection[img_id].append((class_name, bbox[0], bbox[1]))
    return detection

output = {
    "1.nii.gz": [[[50, 50, 50, 150, 150, 150], 0.6, 0.1, 0.9], [[10, 10, 10, 80, 80, 80], 0.5, 0.8, 0.2]],  # Prediction for image 1
    "2.nii.gz": [[[20, 20, 20, 70, 70, 70], 0.4, 0.7, 0.3], [[60, 60, 60, 120, 120, 120], 0.1, 0.4, 0.6]],  # Prediction for image 2
}

print(convert_output_to_detection(output))

{'1.nii.gz': [('stenosis', [50, 50, 50, 150, 150, 150], 0.6), ('IA', [10, 10, 10, 80, 80, 80], 0.5)], '2.nii.gz': [('IA', [20, 20, 20, 70, 70, 70], 0.4), ('stenosis', [60, 60, 60, 120, 120, 120], 0.1)]}


## Segmentation
Output of the segmentation task should be **an array with the same size as the input image**, with label 1 as segmented lesion (no matter whether intracranial aneurysm or stenosis). Patches in successfully detected bounding boxes will be cropped and segmentation metrics and clinical metrics will be calculated within the box.

# Detection Part
Definitions of Metrics

In [2]:
# Using metrics from https://github.com/autonomousvision/kitti360Scripts/blob/master/kitti360scripts/evaluation/semantic_3d/evalDetection.py
def box3dVolume(corners):
    ''' corners: (8,3) no assumption on axis direction '''
    a = np.sqrt(np.sum((corners[0,:] - corners[1,:])**2))
    b = np.sqrt(np.sum((corners[1,:] - corners[2,:])**2))
    c = np.sqrt(np.sum((corners[0,:] - corners[4,:])**2))
    return a*b*c


def roty(t):
    """Rotation about the y-axis."""
    c = np.cos(t)
    s = np.sin(t)
    return np.array([[c,  0,  s],
                    [0,  1,  0],
                    [-s, 0,  c]])

def rotz(t):
    """Rotation about the y-axis."""
    c = np.cos(t)
    s = np.sin(t)
    return np.array([[c,  -s,  0],
                    [s,  c,  0],
                    [0, 0,  1]])


def vocAP(rec, prec, use_07_metric=False):
    """ ap = vocAP(rec, prec, [use_07_metric])
    Compute VOC AP given precision and recall.
    If use_07_metric is true, uses the
    VOC 07 11 point method (default:False).
    """
    if use_07_metric:
        # 11 point metric
        ap = 0.
        for t in np.arange(0., 1.1, 0.1):
            if np.sum(rec >= t) == 0:
                p = 0
            else:
                p = np.max(prec[rec >= t])
            ap = ap + p / 11.
    else:
        # correct AP calculation
        # first append sentinel values at the end
        mrec = np.concatenate(([0.], rec, [1.]))
        mpre = np.concatenate(([0.], prec, [0.]))

        # compute the precision envelope
        for i in range(mpre.size - 1, 0, -1):
            mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

        # to calculate area under PR curve, look for points
        # where X axis (recall) changes value
        i = np.where(mrec[1:] != mrec[:-1])[0]

        # and sum (\Delta recall) * prec
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap


def get3dIoU(box1, box2):
    """
    Calculate Intersection over Union (IoU) between two 3D bounding boxes.
    box1 and box2 should be in the format [x1, y1, z1, x2, y2, z2]
    """
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    z1_inter = max(box1[2], box2[2])
    x2_inter = min(box1[3], box2[3])
    y2_inter = min(box1[4], box2[4])
    z2_inter = min(box1[5], box2[5])

    if x1_inter < x2_inter and y1_inter < y2_inter and z1_inter < z2_inter:
        inter_volume = (x2_inter - x1_inter) * (y2_inter - y1_inter) * (z2_inter - z1_inter)
    else:
        inter_volume = 0

    box1_volume = (box1[3] - box1[0]) * (box1[4] - box1[1]) * (box1[5] - box1[2])
    box2_volume = (box2[3] - box2[0]) * (box2[4] - box2[1]) * (box2[5] - box2[2])

    iou_value = inter_volume / (box1_volume + box2_volume - inter_volume)
    return iou_value

def evalDetectionClass(pred, gt, ovthresh=0.25, use_07_metric=False):
    """ Generic functions to compute precision/recall for object detection
        for a single class.
        Input:
            pred: map of {img_id: [(bbox, score)]} where bbox is numpy array
            gt: map of {img_id: [bbox]}
            ovthresh: scalar, iou threshold
            use_07_metric: bool, if True use VOC07 11 point method
        Output:
            rec: numpy array of length nd
            prec: numpy array of length nd
            ap: scalar, average precision
    """

    # construct gt objects
    class_recs = {} # {img_id: {'bbox': bbox list, 'det': matched list}}
    npos = 0
    for img_id in gt.keys():
        bbox = np.array(gt[img_id])
        det = [False] * len(bbox)
        npos += len(bbox)
        class_recs[img_id] = {'bbox': bbox, 'det': det}

        # import matplotlib.pyplot as plt
        # from mpl_toolkits.mplot3d import Axes3D
        # fig = plt.figure()
        # ax = fig.add_subplot(111, projection='3d')
        # for bbox_ in bbox:
        #     ax.plot(*bbox_.T,'.')
        # ax.set_zlim3d([0,200])
        # plt.show()
    # pad empty list to all other imgids
    for img_id in pred.keys():
        if img_id not in gt:
            class_recs[img_id] = {'bbox': np.array([]), 'det': []}

    # construct dets
    image_ids = []
    confidence = []
    BB = []
    for img_id in pred.keys():
        for box,score in pred[img_id]:
            image_ids.append(img_id)
            confidence.append(score)
            BB.append(box)
    confidence = np.array(confidence)
    BB = np.array(BB) # (nd,4 or 8,3 or 6)

    # sort by confidence
    sorted_ind = np.argsort(-confidence, kind="stable")  # the stable sort method is used to keep the order of the boxes with the same confidence
    sorted_scores = np.sort(-confidence, kind="stable")
    BB = BB[sorted_ind, ...]
    image_ids = [image_ids[x] for x in sorted_ind]

    # go down dets and mark TPs and FPs
    nd = len(image_ids)
    tp = np.zeros(nd)
    fp = np.zeros(nd)
    for d in range(nd):
        R = class_recs[image_ids[d]]
        bb = BB[d,...].astype(float)
        ovmax = -np.inf
        BBGT = R['bbox'].astype(float)

        if BBGT.size > 0:
            # compute overlaps
            for j in range(BBGT.shape[0]):
                iou = get3dIoU(bb, BBGT[j,...])
                if iou > ovmax:
                    ovmax = iou
                    jmax = j

        if ovmax > ovthresh:
            if not R['det'][jmax]:
                tp[d] = 1.
                R['det'][jmax] = 1
            else:
                fp[d] = 1.
        else:
            fp[d] = 1.

    # compute precision recall
    fp = np.cumsum(fp)
    tp = np.cumsum(tp)
    rec = tp / float(npos)
    # avoid divide by zero in case the first detection matches a difficult
    # ground truth
    prec = tp / np.maximum(tp + fp, np.finfo(np.float64).eps)
    ap = vocAP(rec, prec, use_07_metric)

    return rec, prec, ap

def evalDetectionClassWrapper(arguments):
    pred, gt, ovthresh, use_07_metric = arguments
    rec, prec, ap = evalDetectionClass(pred, gt, ovthresh, use_07_metric)
    return (rec, prec, ap)

def evalDetection(pred_all, gt_all, ovthresh=0.25, use_07_metric=False):
    """ Generic functions to compute precision/recall for object detection
        for multiple classes.
        Input:
            pred_all: map of {img_id: [(classname, bbox, score)]}
            gt_all: map of {img_id: [(classname, bbox)]}
            ovthresh: scalar, iou threshold
            use_07_metric: bool, if true use VOC07 11 point method
        Output:
            rec: {classname: rec}
            prec: {classname: prec_all}
            ap: {classname: scalar}
    """
    pred = {} # map {classname: pred}
    gt = {} # map {classname: gt}
    for img_id in pred_all.keys():
        for classname, bbox, score in pred_all[img_id]:
            if classname not in pred: pred[classname] = {}
            if img_id not in pred[classname]:
                pred[classname][img_id] = []
            if classname not in gt: gt[classname] = {}
            if img_id not in gt[classname]:
                gt[classname][img_id] = []
            pred[classname][img_id].append((bbox,score))
    for img_id in gt_all.keys():
        for classname, bbox in gt_all[img_id]:
            if classname not in gt: gt[classname] = {}
            if img_id not in gt[classname]:
                gt[classname][img_id] = []
            gt[classname][img_id].append(bbox)

    rec = {}
    prec = {}
    ap = {}
    for classname in gt.keys():
        print('Computing AP for class: ', classname)
        rec[classname], prec[classname], ap[classname] = evalDetectionClass(pred[classname], gt[classname], ovthresh, use_07_metric)
        print(classname, ap[classname])

    return rec, prec, ap

def convert_gt_to_detection(gt):
    """
    Convert the ground truth to the format of the detection.
    :param gt: {img_id: [[bbox, class_num]]}
    :return: {img_id: [(classname, bbox)]}
    """
    detection = {}
    for img_id, bbox_list in gt.items():
        detection[img_id] = []
        for bbox in bbox_list:
            if bbox[1] == 1:
                class_name = 'IA'
            elif bbox[1] == 2:
                class_name = 'stenosis'
            detection[img_id].append((class_name, bbox[0]))
    return detection

Example Usage

In [7]:
predictions_IA = {  # Predictions for each image in the ENTIRE test set / validation set / training set of ONE class, e.g. aneurysm
    "1.nii.gz": [([50, 50, 50, 150, 150, 150], 0.6, 0.9, 0.1), ([10, 10, 10, 80, 80, 80], 0.5, 0.9, 0.1)],  # Prediction for image 1
    "2.nii.gz": [([20, 20, 20, 70, 70, 70], 0.4, 0.9, 0.1), ([60, 60, 60, 120, 120, 120], 0.1, 0.9, 0.1)],  # Prediction for image 2
    "3.nii.gz": [([40, 40, 40, 160, 160, 160], 0.7, 0.9, 0.1), ([35, 35, 35, 105, 105, 105], 0.2, 0.9, 0.1)],  # Prediction for image 3
}
ground_truths_IA = {  # Ground truths for each image in the ENTIRE test set / validation set / training set of ONE class, e.g. aneurysm
    "1.nii.gz": [[[40, 40, 40, 160, 160, 160], 1], [[35, 35, 35, 105, 105, 105], 1]],  # Ground truth for image 1
    "2.nii.gz": [[[15, 15, 15, 75, 75, 75], 1], [[65, 65, 65, 130, 130, 130], 1]],  # Ground truth for image 2
    "3.nii.gz": [[[50, 50, 50, 150, 150, 150], 1], [[10, 10, 10, 80, 80, 80], 1]],  # Ground truth for image 3
}

bbox_pred = convert_output_to_detection(predictions_IA)
bbox_gt = convert_gt_to_detection(ground_truths_IA)

_, _, ap15 = evalDetection(bbox_pred, bbox_gt, ovthresh=0.15, use_07_metric=False)
_, _, ap25 = evalDetection(bbox_pred, bbox_gt, ovthresh=0.25, use_07_metric=False)
ap15_IA = ap15["IA"]
ap25_IA = ap25["IA"]

ap_IA = (ap15_IA + ap25_IA) / 2

print(ap_IA)

Computing AP for class:  IA
IA 1.0
Computing AP for class:  IA
IA 0.5694444444444444
0.7847222222222222


# Segmentation Part
Definition of Metrics

In [17]:
import numpy as np
from medpy.metric.binary import hd95

def dice_score(pred, gt):
    """
    Calculate Dice score between two binary masks
    """
    pred = np.bool_(pred)
    gt = np.bool_(gt)
    intersection = np.count_nonzero(pred & gt)
    union = np.count_nonzero(pred | gt)
    dice = 2 * intersection / (np.count_nonzero(pred) + np.count_nonzero(gt))
    return dice

def hausdorff_distance_unified(pred, gt, baseline, voxel_spacing):
    """
    Calculate Hausdorff distance between two binary masks, then unify the result to (0-1) with a baseline
    """
    pred = np.bool_(pred)
    gt = np.bool_(gt)
    hd = hd95(pred, gt, voxel_spacing)
    hd_baseline = hd95(baseline, gt, voxel_spacing)
    hd = 1 - hd / hd_baseline
    if hd < 0:
        hd = 0
    return hd

Example Usage

In [20]:
# each metric of one type of lesion is calculated for each lesion (i.e. each bounding box), then averaged across all lesions
dice_scores = []
hd_scores = []

# Example: 160*200*200 image, spacing is (0.8, 0.6, 0.6), lesion ground-truth bounding box at [x1, y1, z1, x2, y2, z2] = [30, 30, 30, 50, 50, 50]
# Note that a predicted bounding box with IoU > 0.15 and any positive probability is considered a true positive
label_img = np.zeros((160, 200, 200))
label_img[30:50, 30:50, 30:50] = 1
pred_img = np.zeros((160, 200, 200))
pred_img[32:48, 31:49, 34:45] = 1
baseline_pred_img = np.zeros((160, 200, 200))  # baseline prediction made by simple thresholding
baseline_pred_img[33:47, 32:48, 36:42] = 1
spacing = (0.8, 0.6, 0.6)

bbox_gt = [30, 30, 30, 50, 50, 50]

pred_img_in_bbox = pred_img[bbox_gt[0]:bbox_gt[3], bbox_gt[1]:bbox_gt[4], bbox_gt[2]:bbox_gt[5]]
label_img_in_bbox = label_img[bbox_gt[0]:bbox_gt[3], bbox_gt[1]:bbox_gt[4], bbox_gt[2]:bbox_gt[5]]
baseline_pred_img_in_bbox = baseline_pred_img[bbox_gt[0]:bbox_gt[3], bbox_gt[1]:bbox_gt[4], bbox_gt[2]:bbox_gt[5]]

dice = dice_score(pred_img_in_bbox, label_img_in_bbox)
hd = hausdorff_distance_unified(pred_img_in_bbox, label_img_in_bbox, baseline_pred_img_in_bbox, spacing)
dice_scores.append(dice)
hd_scores.append(hd)

total_dice = np.mean(dice_scores)
total_hd = np.mean(hd_scores)

print(f'Dice: {total_dice}, HD: {total_hd}')


Dice: 0.5673352435530086, HD: 0.38350379760491016


# Clinical Part
## Stenosis percentage
Definition of Metrics

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, measure, morphology
from scipy.ndimage import distance_transform_edt

def get_max_connective_field(cube):
    if np.sum(cube) == 0:
        return cube
    result = measure.label(cube)
    result1 = result.reshape([-1])
    lst = np.bincount(result1)
    lst[0] = 0
    a = np.argmax(lst)
    result[result != a] = 0
    result[result == a] = 1
    return result

def max_and_min_diameters(segmentation_image, spacing):
    """
    :param segmentation_image: Binary segmentation image with the vessel as 1 and background as 0.
    """
    binary_image = segmentation_image > 0
    binary_image = get_max_connective_field(binary_image)  # The max connective field of the segmentation result is picked
    skeleton = morphology.skeletonize(binary_image)
    distance_transform = distance_transform_edt(binary_image, sampling=spacing)

    # Get the coordinates of the skeleton
    skeleton_coords = np.column_stack(np.where(skeleton))

    # Calculate the diameter at each point in the skeleton
    diameters = [2 * distance_transform[tuple(coord)] for coord in skeleton_coords]

    return np.max(diameters), np.min(diameters)

Example Usage

In [8]:
from skimage.morphology import disk

def generate_example_image(disk_radii):
    image = np.zeros([len(disk_radii), 50, 50])
    for i, radius in enumerate(disk_radii):
        image[i, 25 - radius:25 + radius + 1, 25 - radius:25 + radius + 1] = disk(radius)

    return image

spacing = (0.8, 0.6, 0.6)

gt_label = generate_example_image([5, 5, 5, 4, 3, 2, 3, 4, 5, 5, 5])  # diameter of ordinary vessel is 9 and stenosis is 5
pred_label = generate_example_image([4, 3, 2, 1, 2, 3, 4])  # predicted diameter of ordinary vessel is 9 and stenosis is 3
gt_max, gt_min = max_and_min_diameters(gt_label, spacing)  # note that we have labelled the ordinary vessel beside the stenosis site in our test set GT using another label, so we can calculate the diameter of the ordinary vessel
_, pred_min = max_and_min_diameters(pred_label, spacing)
gt_percentage = (gt_max - gt_min) / gt_max
pred_percentage = (gt_max - pred_min) / gt_max
print(f'Ground truth vessel percentage: {gt_percentage}, Predicted vessel percentage: {pred_percentage}, Difference: {abs(gt_percentage - pred_percentage)}')

Ground truth vessel percentage: 0.45943752238266466, Predicted vessel percentage: 0.6581182706210862, Difference: 0.19868074823842152


The final score of this part is _1 - Difference_. Values lower than 0 will be set to 0.

## Aneurysm long and short axes length
Definition of Metrics

In [8]:
import os
from tqdm import tqdm
from skimage import measure
from scipy.spatial.distance import pdist, squareform
import numpy as np

def max_diameter_short_radius(arr):
    flat_arr = np.ravel(arr)
    indices = np.where(flat_arr == 1)[0]
    coordinates = np.column_stack(np.unravel_index(indices, arr.shape))
    distances = squareform(pdist(coordinates))
    i, j = np.unravel_index(np.argmax(distances), distances.shape)
    max_diameter = distances[i, j]
    midpoint = (coordinates[i] + coordinates[j]) / 2
    vector = coordinates[j] - coordinates[i]
    perp_vector = np.array([-vector[1], vector[0]])
    k = np.argmax(np.abs(np.dot(coordinates - midpoint, perp_vector)))
    l = np.argmin(np.abs(np.dot(coordinates - midpoint, perp_vector)))
    short_radius = np.linalg.norm(coordinates[k] - coordinates[l])
    return max_diameter, short_radius

def get_2d_diameters(label_arr,nodule_spacing):
    mask = label_arr
    mask = get_max_connective_field(mask)  # The max connective field of the segmentation result is picked
    largest_z = np.argmax(np.sum(mask, axis=(1,2)))
    lag_z = mask[largest_z]
    max_diameter, short_diameter =max_diameter_short_radius(lag_z)
    return max_diameter*nodule_spacing[1],short_diameter*nodule_spacing[1]

Example Usage

In [9]:
from skimage.morphology import ball
import numpy as np

def generate_example_image(disk_radius):
    i = np.zeros([100, 100, 100])
    b = ball(disk_radius)
    i[50:50 + b.shape[0], 50:50 + b.shape[1], 50:50 + b.shape[2]] = b
    return i

spacing = (0.8, 0.6, 0.6)

gt_label = generate_example_image(5)
pred_label = generate_example_image(4)
bbox = [50, 50, 50, 50 + 2 * 5 + 1, 50 + 2 * 5 + 1, 50 + 2 * 5 + 1]

gt_label_in_bbox = gt_label[bbox[0]:bbox[3], bbox[1]:bbox[4], bbox[2]:bbox[5]]
pred_label_in_bbox = pred_label[bbox[0]:bbox[3], bbox[1]:bbox[4], bbox[2]:bbox[5]]

gt_max_diameter, gt_short_diameter = get_2d_diameters(gt_label, spacing)
pred_max_diameter, pred_short_diameter = get_2d_diameters(pred_label, spacing)
print(f'Ground truth max diameter: {gt_max_diameter}, Predicted max diameter: {pred_max_diameter}, NMAE: {abs(gt_max_diameter - pred_max_diameter) / gt_max_diameter}')
print(f'Ground truth short diameter: {gt_short_diameter}, Predicted short diameter: {pred_short_diameter}, NMAE: {abs(gt_short_diameter - pred_short_diameter) / gt_short_diameter}')



Ground truth max diameter: 6.0, Predicted max diameter: 4.8, NMAE: 0.20000000000000004
Ground truth short diameter: 4.242640687119285, Predicted short diameter: 3.394112549695428, NMAE: 0.19999999999999993


The final score of this part is _1 - NMAE_. Values lower than 0 will be set to 0.